In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
# -----------------------
#ordenando el data set 
# -----------------------
def load_data():
    #Carga del dataset original
    df = pd.read_csv("C:/Users/k_lei/Documents/america_crop_production/america_crop_production/Production_Crops_E_Americas.csv", encoding='latin-1') 

    # Filtrado de datos por Elemento clave (en el DF hay una fila para cada uno)
    production = df[df['Element'] == 'Production'].copy()
    area = df[df['Element'] == 'Area harvested'].copy()
    yield_ = df[df['Element'] == 'Yield'].copy()

    # Extraer solo columnas de años (ignorando flags)
    years = [col for col in df.columns if col.startswith('Y') and not col.endswith('F')]

    # Transformar a formato largo
    def melt_element(df, element_name):
        df_long = df.melt(
            id_vars=['Area','Item','Element'],      # columnas que identifican la fila
            value_vars=years,                       # columnas de datos
            var_name='Year',                        # nueva columna con el año
            value_name='Value'                   # nombre de la columna con valores
        )

        df_long['Year'] = df_long['Year'].str[1:].astype(int)
        df_long['Value'] = pd.to_numeric(df_long['Value'], errors='coerce')
        df_long['Element'] = element_name  
        return df_long

    # Convertir tres elementos a formato largo
    production_long = melt_element(production, "Production")
    area_long = melt_element(area, "Area harvested")
    yield_long = melt_element(yield_, "Yield")
    
    # Unir dataframes
    df_long = pd.concat([production_long, area_long, yield_long], ignore_index=True)
    
    # Limpiar
    df_long.dropna(subset=["Value"], inplace=True)
    df_long.drop_duplicates(inplace=True)
    
    # Renombrar para estética
    df_long.rename(columns={"Area": "Country"}, inplace=True)

    return df_long

df_long = load_data()

# ------------------------------------------------------
# Configuración inicial de la app
# ------------------------------------------------------
st.title("Exploración de Producción Agrícola en América")
st.write("Visualización interactiva de datos FAO: producción, área cosechada y rendimiento.")


# Panel de filtros
st.sidebar.header("Filtros")
country = st.sidebar.selectbox("Selecciona un país:", sorted(df_long["Country"].unique()))
crop = st.sidebar.selectbox("Selecciona un cultivo:", sorted(df_long["Item"].unique()))
year = st.sidebar.slider("Selecciona un año", min_value=int(df_long["Year"].min()), max_value=int(df_long["Year"].max()), value=int(df_long["Year"].max()))
element = st.sidebar.selectbox(
    "Variable a visualizar:",
    ["Production", "Area harvested"]
)

# Filtro aplicado
filtered = df_long[
    (df_long["Country"] == country) &
    (df_long["Item"] == crop) &
    (df_long["Element"] == element) &
    (df_long["Year"] == year)
]

st.subheader(f"Histograma para {crop} en {country} ({year})")
st.caption(f"Variable: {element}")

if st.button("Mostrar histograma"):
    if filtered.empty:
        st.warning("No hay datos para esta combinación. Prueba con otros filtros.")
    else:
        fig = px.histogram(filtered, x="Value", nbins=10, title="Distribución de valores")
        st.plotly_chart(fig, use_container_width=True)

In [ ]:
#Carga del dataset
crop_data = pd.read_csv("C:/Users/k_lei/Documents/america_crop_production/america_crop_production/Production_Crops_E_Americas.csv", encoding='latin-1') 

#Histograma
fig = px.histogram(crop_data, x="Area") # crear un histograma
fig.show() # crear gráfico de dispersión

In [25]:
fig = px.histogram(crop_data, x="Item") # crear un histograma
fig.show() # crear gráfico de dispersión

Visualizaciones de exploración

In [ ]:
# Filtrar datos por Elemento
production = crop_data[crop_data['Element'] == 'Production'].copy()
area = crop_data[crop_data['Element'] == 'Area harvested'].copy()
yield_ = crop_data[crop_data['Element'] == 'Yield'].copy()


In [16]:
# Extraer solo columnas de años (ignorando flags)
# -----------------------
years = [col for col in production.columns if col.startswith('Y') and not col.endswith('F')]

# -----------------------
# Transformar a formato largo
# -----------------------
def melt_element(df, value_name):
    df_long = df.melt(
        id_vars=['Area','Item'],      # columnas que identifican la fila
        value_vars=years,             # columnas de datos
        var_name='Year',              # nueva columna con el año
        value_name=value_name         # nombre de la columna con valores
    )
    df_long['Year'] = df_long['Year'].str[1:].astype(int)
    df_long[value_name] = pd.to_numeric(df_long[value_name], errors='coerce')
    return df_long

production_long = melt_element(production, 'Production')
area_long = melt_element(area, 'Area_ha')
yield_long = melt_element(yield_, 'Yield_hg_per_ha')


In [23]:
# -----------------------
# Tendencia temporal de producción (ejemplo: Bananas en Costa Rica)
# -----------------------
prod_subset = production_long[
    (production_long['Area'] == 'Costa Rica') &
    (production_long['Item'] == 'Avocados')
]

fig1 = px.line(
    prod_subset, 
    x='Year', y='Production', 
    title='Producción de Aguacates en Costa Rica',
    markers=True
)
fig1.show()


In [18]:
# -----------------------
# Scatter: Área vs Yield (promedio de todos los años)
# -----------------------
# Promedio por país y cultivo
area_avg = area_long.groupby(['Area','Item'])['Area_ha'].mean()
yield_avg = yield_long.groupby(['Area','Item'])['Yield_hg_per_ha'].mean()

df_scatter = pd.DataFrame({
    'Area_avg': area_avg,
    'Yield_avg': yield_avg
}).reset_index()

fig2 = px.scatter(
    df_scatter,
    x='Area_avg',
    y='Yield_avg',
    color='Item',
    hover_data=['Area'],
    title='Relación Área cosechada vs Rendimiento'
)
fig2.show()



In [19]:
# -----------------------
# Producción promedio por país
# -----------------------
prod_avg = production_long.groupby('Area')['Production'].mean().sort_values(ascending=False)
df_bar = prod_avg.reset_index().rename(columns={'Production':'Production_avg'})

fig3 = px.bar(
    df_bar,
    x='Area',
    y='Production_avg',
    title='Producción promedio total por país'
)
fig3.show()